In [13]:
import tensorflow as tf
from tensorflow import keras
from keras import layers

In [25]:
import os
dirs = 'images'

for dir in os.listdir(dirs):
    for image_name in os.listdir(os.path.join(dirs,dir)):
        if image_name.endswith('.jpeg') or image_name.endswith('.jpg') or image_name.endswith('.png') or image_name.endswith('.bmp'):
            continue
        else:
            print(image_name)
            os.remove(os.path.join(dirs, dir, image_name))

In [26]:
image_size = (200, 200)
batch_size = 128

train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
    "images",
    validation_split=0.2,
    subset="both",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

Found 439 files belonging to 3 classes.
Using 352 files for training.
Using 87 files for validation.


In [27]:
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
    ]
)

In [28]:
# Apply `data_augmentation` to the training images.
train_ds = train_ds.map(
    lambda img, label: (data_augmentation(img), label),
    num_parallel_calls=tf.data.AUTOTUNE,
)
# Prefetching samples in GPU memory helps maximize GPU utilization.
train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

In [29]:
def make_model(input_shape, num_classes):
    inputs = keras.Input(shape=input_shape)

    # Entry block
    x = layers.Rescaling(1.0 / 255)(inputs)
    x = layers.Conv2D(128, 3, strides=2, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [256, 512, 728]:
        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.Activation("relu")(x)
        x = layers.SeparableConv2D(size, 3, padding="same")(x)
        x = layers.BatchNormalization()(x)

        x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation("relu")(x)

    x = layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(units, activation=activation)(x)
    return keras.Model(inputs, outputs)


model = make_model(input_shape=image_size + (3,), num_classes=2)

In [30]:
epochs = 25

model.compile(
    optimizer=keras.optimizers.Adam(1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"],
)
model.fit(
    train_ds,
    epochs=epochs,
    validation_data=val_ds,
)

Epoch 1/25
3/3 [==============================] - 570s 169s/step - loss: 0.1912 - accuracy: 0.2472 - val_loss: 0.6201 - val_accuracy: 0.3448
Epoch 2/25
3/3 [==============================] - 513s 161s/step - loss: -1.6789 - accuracy: 0.3381 - val_loss: 0.5401 - val_accuracy: 0.3448
Epoch 3/25
3/3 [==============================] - 517s 169s/step - loss: -2.8851 - accuracy: 0.3381 - val_loss: 0.4476 - val_accuracy: 0.3448
Epoch 4/25
3/3 [==============================] - 496s 156s/step - loss: -4.1182 - accuracy: 0.3295 - val_loss: 0.3351 - val_accuracy: 0.3448
Epoch 5/25
3/3 [==============================] - 503s 163s/step - loss: -5.1744 - accuracy: 0.3381 - val_loss: 0.2156 - val_accuracy: 0.3448
Epoch 6/25
3/3 [==============================] - 497s 155s/step - loss: -6.2883 - accuracy: 0.3210 - val_loss: 0.0704 - val_accuracy: 0.3448
Epoch 7/25
3/3 [==============================] - 515s 161s/step - loss: -7.1771 - accuracy: 0.3295 - val_loss: -0.0890 - val_accuracy: 0.3448
Epoch 

In [31]:
model.save('model_hair_classifier.h5')

c:\Users\alish\Anaconda3\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
